# Overview

This notebook extracts unstructured data from Microsoft Outlook emails regarding Travel Waivers
that United Airlines sends out related to events/weather/etc. that may impact travel. 

The email data is cleaned and manipulated into a DataFrame that includes the Travel Waiver's
date sent, event name, severity level, cities impacted, and dates affected. 

The code can be broken down into the following 3 sections:

    1. Scrape Outlook to collect all Travel Waiver emails
    2. Create functions used to parse the email's body text 
    3. Iterate over the data set to clean and structure data

Over time, the collection of this data may yield some interesting insights into events that impact travel.
Feel free to ask questions or leave suggestions/critique, I love to learn other's approaches to problems!

### The image below is an example of the email being scraped and my mailbox had over a year's worth of Travel Waiver emails 

![TravelWaiverImage](https://raw.githubusercontent.com/eli64s/Python-Email-Scraper/master/travel_waiver.PNG)

#### Use these strings to test the notebook on your own!

In [1]:
# The following strings come from the example Travel Waiver email above  
email_subject_example = 'Travel Waiver (SEV2) Typhoon Lingling - 09/05/19'
email_body_example = """Travel Waiver: Typhoon Lingling
SEV 2
We have implemented a travel waiver for Seoul, South Korea due to Typhoon Lingling.
*    Cities: ICN
*    Travel date(s): September 6 - 7, 2019
*	Must be ticketed by: September 4, 2019
Avoid affected city on connections, if possible.
Permitted changes
*      Same origin and destination
*      Any booking code in originally ticketed cabin
*      Different connection
*      Rebook only permitted on UA/UAX 
Standby
Same day confirmed 
Alternate day confirmed
*    Rescheduled travel up to/including September 14, 2019
Reuse ticket or coupon(s)
*    Use full value of unused ticket/coupon(s) for another itinerary within ticket validity
*    New itinerary can be any cities and cabin
*    Subject to add-collect of new fare
What’s waived?
Standby fee
Change fee: Same or alternate dates listed above
Add-collect: Within rescheduled travel dates listed above
Travel agency waiver code: 7JC5U 
ATRE and DRS: TRAVEL WAIVER have been updated. United.com will be updated shortly.
Reminder: Flights that are already canceled or delayed fall under the Irregular Operations policy.
Goal: Same day travel flexibility | Close-in change at no cost |
Use full value of unused ticket or coupon(s) for another itinerary by September 4, 2020"""

# Outlook Setup

In [2]:
import pandas as pd
import win32com.client
 
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6) # '6' refers to the index of a folder - in this case the Inbox Folder
messages = inbox.Items              # Get all items in the Inbox folder

### Retrieve Mailbox Messages

#### Iterate over the messages in the mailbox, storing the extracted data in a Dictionary 

In [3]:
travel_waivers = {} # Dictionary{} to store email data - send date, title, body text
key_val = 0         # key value for Dictionary{}

for message in messages:
    '''
    Iterate over messages to collect data from each email belonging to
    SenderName - 'Travel Waivers'
    Try/Except statements are used as some emails have an unknown SenderName
    '''
    
    try:
        if message.SenderName == 'Travel Waivers':
            
            # Condition is True when the email is a reply to another email
            if 'From:' in message.Body:
                pass
            
            # Insert Travel Waiver data into Dictionary{}
            else:
                travel_waivers[key_val] = {
                    'email_title': message.Subject,
                    'email_body': message.Body,
                    'Date_Sent': message.SentOn.strftime('%Y-%m-%d'),
                    'Event': None,
                    'Indicator': None,
                    'City': None,
                    'Start_Date': None,
                    'End_Date': None,
                    'All_Dates': None
                }
                key_val += 1
            
    # Exception occurs when SenderName is unknown
    except Exception as error_message:
        print(error_message)

<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName


# Create Functions to Parse Text of Email Body

#### Function: Returns event name and severity indicator from the email title

In [4]:
import re

def clean_title(text):
    '''
    Function that takes the email subject and extracts the 
    Travel Waiver event name and severity indicator 
    '''
    
    # Email subjects are uniform, with the event name following ')' 
    event_name = text.split(')')[1]    
    event_name = event_name.split('-')[0].strip()
    
    # All severity indicators start with '(S'  - example: 'SEV1' 
    ind = text.split()
    ind = [i for i in ind if i.startswith('(S')][0]
    ind = re.sub('[()]', '', ind)
    
    return event_name, ind

#### Example output of clean_title() function using the provided email subject example

In [5]:
print('Original Email Subject: {}\n\nResult:'.format(email_subject_example))
print('Event Name: {}'.format(clean_title(email_subject_example)[0]))
print('Indicator: {}'.format(clean_title(email_subject_example)[1]))

Original Email Subject: Travel Waiver (SEV2) Typhoon Lingling - 09/05/19

Result:
Event Name: Typhoon Lingling
Indicator: SEV2


####  Function: Returns a substring between two words using regular expressions

In [6]:
def string_search(text, start_word, end_word):
    '''
    This function returns a substring between two words 
    using regular expressions
    '''
    
    # 'c' and 'd' are arbitrary arguments as placeholders 
    # This condition extracts substring of city codes in the Travel Waiver 
    if start_word == 'c':
        # Search for substring following City and preceding Travel date
        start_word = re.findall(r"City:|Cities:", text)[0]              
        end_word = re.findall(r"Travel date:|Travel date\(s\):|Travel dates:", text)[0]

    # This condition extracts substring of dates in the Travel Waiver
    if end_word == 'd':
        start_word = re.findall(r"Travel date:|Travel date\(s\):|Travel dates:", text)[0]
        end_word = re.findall(', \d{4}', text)[0]   # Regex to find the ', YYYY'
        end_word = text.split(end_word, 1)[1]       # Gets entire string after ', YYYY'

    # Use starting and ending word to extract substring with desired data  
    substring_data = text[text.find(start_word) + len(start_word): text.rfind(end_word)]
   
    return substring_data

#### Example output of string_search() function using the provided email body example

In [7]:
print('Original Text:\n\n {}\n\nResult:'.format(email_body_example))
print('City substring: {}'.format(string_search(email_body_example, 'c', 'c')))
print('Date substring: {}'.format(string_search(email_body_example, 'd', 'd')))

Original Text:

 Travel Waiver: Typhoon Lingling
SEV 2
We have implemented a travel waiver for Seoul, South Korea due to Typhoon Lingling.
*    Cities: ICN
*    Travel date(s): September 6 - 7, 2019
*	Must be ticketed by: September 4, 2019
Avoid affected city on connections, if possible.
Permitted changes
*      Same origin and destination
*      Any booking code in originally ticketed cabin
*      Different connection
*      Rebook only permitted on UA/UAX 
Standby
Same day confirmed 
Alternate day confirmed
*    Rescheduled travel up to/including September 14, 2019
Reuse ticket or coupon(s)
*    Use full value of unused ticket/coupon(s) for another itinerary within ticket validity
*    New itinerary can be any cities and cabin
*    Subject to add-collect of new fare
What’s waived?
Standby fee
Change fee: Same or alternate dates listed above
Add-collect: Within rescheduled travel dates listed above
Travel agency waiver code: 7JC5U 
ATRE and DRS: TRAVEL WAIVER have been updated. United

#### Function: Returns a list of the city codes in the given Travel Waiver 

In [8]:
def clean_cities(text):
    '''
    This function takes the substring of city codes
    that was created from the string_search() function  
    Function returns of a list of each 3 character city code
    '''
    
    text.strip('/')                                          # Strip '/' that separates each city code
    city_text = ''.join(ch for ch in text if ch.isalnum())        # Delete non-alphanumeric characters
    city_text = [city_text[i:i + 3] for i in range(0, len(city_text), 3)]  # Create list of city codes
                                                                    
    return city_text

#### Example output of clean_cities() function

In [9]:
'''
This function example will not use the provided example data
as only 1 city was impacted in that Travel Waiver. We want to test this
function on a longer list of cities. The following city substring
was returned from the string_search() function
'''
city_substring_example = '*   CHS / FAY / HHH / ILM / MYR / SAV    *'
print('City Substring: {}\n\nResult:'.format(city_substring_example))
print('City Codes: {}'.format(clean_cities(city_substring_example)))

City Substring: *   CHS / FAY / HHH / ILM / MYR / SAV    *

Result:
City Codes: ['CHS', 'FAY', 'HHH', 'ILM', 'MYR', 'SAV']


#### Function: Returns the issued date range for the Travel Waiver

In [10]:
from datetime import datetime, timedelta
import calendar

# Generate a list of calendar months name & number to match against dates in waiver
# List[Tuple()] then converted to Dictionary{}
month_names = [(calendar.month_name[month], n + 1) for n, month in enumerate(range(1,13))]
month_names = dict(month_names)


def get_dates(text):
    '''
    This function takes the substring from the string_search() function
    to get the Travel Waiver's issued date range   '
    This function will return the start and end date, along with
    a list of every date in that range, all in YYYY-mm-dd format
    '''
    
    months = [m for m in month_names if m in text]          # Travel Waiver's month(s) 
    day_and_year = list(map(int, re.findall(r'\d+', text))) # Regex to get numeric values
    year = day_and_year[-1]                                 # Travel Waiver's year 

    # Condition for Travel Waivers spanning multiple days: ex) 'August 25 - 27'
    if len(day_and_year) == 3 and len(months) == 1:
        mnth = int(month_names[months[0]])
        start_day = day_and_year[0]         # Starting day
        end_day = day_and_year[1]           # Ending day

        day_range = (datetime(year, mnth, end_day) - datetime(year, mnth, start_day)).days
        date_range_list = [(datetime(year, mnth, start_day) + \
                    timedelta(days = d)).strftime('%Y-%m-%d') for d in range(day_range + 1)]
        start_date = date_range_list[0]     # Start date in the date range
        end_date = date_range_list[-1]      # Last date in the date range

        
    # Condition for Travel Waivers spanning multiple months: ex) 'September 30 – October 2, 2019'
    elif len(day_and_year) == 3 and len(months) == 2:
        start_month = int(month_names[months[0]])   # Starting month number 
        end_month = int(month_names[months[1]])     # Ending month number
        start_day = day_and_year[0]                 # Starting day
        end_day = day_and_year[1]                   # Ending day
                
        day_range = (datetime(year, end_month, end_day) - datetime(year, start_month, start_day)).days
        date_range_list = [(datetime(year, start_month, start_day) + \
                            timedelta(days = d)).strftime('%Y-%m-%d') for d in range(day_range + 1)]
        start_date = date_range_list[0]         # Start date in the date range
        end_date = date_range_list[-1]          # Last date in the date range

        
    # Condition for Travel Waiver that is valid for a single day
    else: 
        mnth = int(month_names[months[0]])
        date_range_list = datetime(year, mnth, day_and_year[0]).strftime('%Y-%m-%d')
        start_date = date_range_list
        end_date = date_range_list
        
        
    return start_date, end_date, date_range_list

#### Example output of get_dates() function using the provided email body example

In [11]:
print('Date Substring: {}\n\nResult:'.format(string_search(email_body_example, 'd', 'd')))
print('Start Date: {}'.format(get_dates(string_search(email_body_example, 'd', 'd'))[0]))
print('End Date: {}'.format(get_dates(string_search(email_body_example, 'd', 'd'))[1]))
print('All Dates: {}'.format(get_dates(string_search(email_body_example, 'd', 'd'))[2]))

Date Substring:  September 6 - 7, 2019

Result:
Start Date: 2019-09-06
End Date: 2019-09-07
All Dates: ['2019-09-06', '2019-09-07']


# Iterate over Dictionary{} to Create the Cleaned Data Set

#### The functions above are called in the iteration below to extract and structure the data 

In [12]:
for value in travel_waivers:
    '''
    Iterate over the Travel Waiver Dictionary{}, calling the functions
    above to clean and structure the data for output
    '''
    
    # Remove the '\r\n' in the email body text
    body_text = travel_waivers[value]['email_body']
    body_text = body_text.replace('\n', ' ').replace('\r', '')

    # Call 'clean_title()' function 
    title_text = travel_waivers[value]['email_title']
    event, indicator = clean_title(title_text)
    
    # Call 'string_search()' function to extract the cities and dates text from the Travel Waiver
    cities = string_search(body_text, 'c', 'c')
    dates = string_search(body_text, 'd', 'd')
    
    # Call 'clean_cities()' function to create list of city codes from the Travel Waiver
    city_list = clean_cities(cities)
    
    # Call 'get_dates()' function to get list of dates (YYYY-mm-dd) associated with the Travel Waiver 
    start, end, all_dates = get_dates(dates)
    
    # Update dictionary values
    travel_waivers[value]['Event'] = event
    travel_waivers[value]['Indicator'] = indicator
    travel_waivers[value]['email_body'] = body_text
    travel_waivers[value]['City'] = city_list
    travel_waivers[value]['Start_Date'] = start
    travel_waivers[value]['End_Date'] = end
    travel_waivers[value]['All_Dates'] = all_dates

# Prepare Final Data for Output

In [13]:
# Convert the Dictionary with the data to DataFrame
travel_waiver_data = pd.DataFrame.from_dict(travel_waivers, orient = 'index')

# Drop the columns that contained the original scraped date ['email_title', 'email_body']
travel_waiver_data = travel_waiver_data[[
    'Date_Sent', 
    'Event',
    'Indicator', 
    'City',
    'Start_Date',
    'End_Date',
    'All_Dates'
    ]]

In [14]:
# explode() function takes a DataFrame column and creates a new row for each item  
travel_waiver_data_expanded = travel_waiver_data.explode('City')
travel_waiver_data_expanded = travel_waiver_data_expanded.explode('All_Dates')
travel_waiver_data_expanded = travel_waiver_data_expanded.drop_duplicates().reset_index(drop = True)

#### Save DataFrame as .csv file

In [15]:
today = datetime.now().strftime('%Y%m%d')
save_path = 'travel_waivers_' + today + '.csv'
travel_waiver_data_expanded.to_csv(save_path, index = False)

In [16]:
travel_waiver_data_expanded.tail(20)

,Date_Sent,Event,Indicator,City,Start_Date,End_Date,All_Dates
1958,2018-12-25,Northern Plains Winter Weather,SEV1+,DLH,2018-12-26,2018-12-27,2018-12-26
1959,2018-12-25,Northern Plains Winter Weather,SEV1+,DLH,2018-12-26,2018-12-27,2018-12-27
1960,2018-12-25,Northern Plains Winter Weather,SEV1+,DVL,2018-12-26,2018-12-27,2018-12-26
1961,2018-12-25,Northern Plains Winter Weather,SEV1+,DVL,2018-12-26,2018-12-27,2018-12-27
1962,2018-12-25,Northern Plains Winter Weather,SEV1+,FAR,2018-12-26,2018-12-27,2018-12-26
1963,2018-12-25,Northern Plains Winter Weather,SEV1+,FAR,2018-12-26,2018-12-27,2018-12-27
1964,2018-12-25,Northern Plains Winter Weather,SEV1+,FSD,2018-12-26,2018-12-27,2018-12-26
1965,2018-12-25,Northern Plains Winter Weather,SEV1+,FSD,2018-12-26,2018-12-27,2018-12-27
1966,2018-12-25,Northern Plains Winter Weather,SEV1+,ISN,2018-12-26,2018-12-27,2018-12-26
1967,2018-12-25,Northern Plains Winter Weather,SEV1+,ISN,2018-12-26,2018-12-27,2018-12-27
